# Análise Geoespacial dos Casos de Varíola do Macaco



O objetivo dessa análise é realizar uma análise geoespecial dos casos de Varíola do Macaco (MonkeyPox) no mundo.

O dataset do número de casos por país foi disponibilizado via <a href="https://www.kaggle.com/datasets/programmerrdai/monkey-pox-next-covid?resource=download">Kaggle</a> e representa os dados no período de 06/maio a 07/agosto.

O arquivo ShapeFile que representa os dados geoespaciais dos países foi disponibilizado pelo <a href="https://hub.arcgis.com/datasets/esri::world-countries-generalized/explore?location=-0.688591%2C0.000000%2C2.03">ArcGIS Hub</a>.

Foram utilizados as bibliotecas <a href="https://pandas.pydata.org/docs/">Pandas</a>, <a href="https://geopandas.org/en/stable/docs.html">GeoPandas</a> e <a href="https://python-visualization.github.io/folium/">Folium</a>.

A biblioteca GeoPandas é utilizada para manipulação de dados geoespaciais enquanto o Folium é utilizado para a visualização dos dados geoespaciais.

Importando as bibliotecas necessárias

In [ ]:
import pandas as pd
import geopandas as gpd
import folium

Lendo base de dados

In [ ]:
df_mp = pd.read_csv('monkeypox.csv')
df_mp.head()

Realizando o agrupamento do número de casos por país, visto que a base informa os novos casos por dia.

In [ ]:
df = df_mp.groupby('location')['new_cases'].sum().to_frame().reset_index()
df

Realizando o shapefile dos países com delimitação da geometria dos países e suas informações.

In [ ]:
geo_df = gpd.read_file('shapefile/World_Countries__Generalized_.shp')
geo_df.plot()

In [ ]:
geo_df.head()

Verificando se temos algum país na base que não possui correspondência no Shapefile.

In [ ]:
df_missing = df.merge(geo_df, how='left', right_on = 'COUNTRY', left_on = 'location')
df_missing = df_missing[df_missing['geometry'].isnull()]
df_missing['location']

Verificando os nomes dos países no Shapefile para realizar o ajuste.

In [ ]:
geo_df['COUNTRY'].unique()

- Para os países Republic of Congo, Russia e Czechia vamos renomeá-los para o nome que corresponde ao Shapefile;
- Como Taiwan é uma província da China, vamos renomear os casos considerando como China para conseguirmos identificar visualmente o dado;
- Vamos deletar os dados agregados (World).

In [ ]:
df.loc[df.location == 'Republic of Congo', 'location'] = 'Congo DRC'
df.loc[df.location == 'Russia', 'location'] = 'Russian Federation'
df.loc[df.location == 'Czechia', 'location'] = 'Czech Republic'
df.loc[df.location == 'Taiwan', 'location'] = 'China'
df.drop(df.loc[df.location == 'World'].index, inplace=True)
df = df.groupby('location')['new_cases'].sum().to_frame().reset_index()

Realizando o merge entre os dois dataframes para trazer a informação do número de casos para o dataframe que possui as geometrias.

In [ ]:
geo_df = geo_df.merge(df, left_on = 'COUNTRY', right_on = 'location')
geo_df.head()

Realizando o plot dos dados via Folium.

No campo de geo_data informamos o dataframe com a geometria no formato json.

No campo data e columns informamos o dataframe com os campos que serão utilizados para legenda.

A própria biblioteca através do padrão de cores (nesse caso utilizado a YlOrRd) realiza uma escala de cores e cria uma legenda que fica no canto superior direito.

In [ ]:
m = folium.Map(min_zoom = 2)

choropleth = folium.Choropleth(
    geo_data=geo_df.to_json(),
    data=geo_df,
    columns=["location", "new_cases"],
    key_on="feature.properties.COUNTRY",
    fill_color="YlOrRd",
    nan_fill_opacity = 0,
    legend_name="Número de Casos"
).add_to(m)

legends = folium.features.GeoJsonTooltip(
                        fields=['location', 'new_cases'],
                        aliases=['País: ', 'Número de Casos: ']
                )

choropleth.geojson.add_child(legends)
    
folium.LayerControl().add_to(m)

m